In [7]:
import torch
import diffusers
import matplotlib.pyplot as plt
from datasets import load_dataset
from torchvision import transforms
import kagglehub
from pandas import read_csv, concat
from pandas import DataFrame

from PIL import Image
import numpy as np

## Data
Download, transform and import images from dataset to program

In [3]:
# Import the Dataset from kaggle and load on a Tensor
Nist_dataset = kagglehub.dataset_download("sachinpatel21/az-handwritten-alphabets-in-csv-format")

chunksize = 250

# the list that contains all the dataframes
list_of_dataframes = []

for df in read_csv(Nist_dataset+"/A_Z Handwritten Data.csv", chunksize=chunksize):
    # process your data frame here
    # then add the current data frame into the list
    list_of_dataframes.append(df)

# if you want all the dataframes together, here it is
handwritten_alphabet = concat(list_of_dataframes)



# handwritten_alphabet = read_csv(Nist_dataset+"/A_Z Handwritten Data.csv")
# print(handwritten_alphabet)
# handwritten_alphabet = torch.Tensor(handwritten_alphabet.values).cuda()

In [9]:

class HandwrittenAlphabetDataset(torch.utils.data.Dataset):
    def __init__(self, Alphabet_dataframe:DataFrame, img_size: int, transform=None):
        """
        Args:
            csv_file (string): Ruta al archivo CSV.
            transform (callable, optional): Transformación opcional para aplicar a la imagen.
        """
        self.data = Alphabet_dataframe
        self.transform = transform
        self.img_size = img_size

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        # Obtener la etiqueta y los píxeles
        label = self.data.iloc[idx, 0]
        pixels = self.data.iloc[idx, 1:].values.astype(np.uint8)

        # Reformar los píxeles en una imagen (ej. 28x28)
        img = pixels.reshape(self.img_size, self.img_size)
        img = Image.fromarray(img) # Convertir a objeto PIL Image

        if self.transform:
            img = self.transform(img)

        return img, label

In [53]:
image_size = 28

#Prepare images to load to the net

preprocess = transforms.Compose(
    [
        transforms.Resize((image_size,image_size)),
        transforms.ToTensor(),              #Convert to Tensor
        transforms.Normalize([0.5],[0.5])   #Map tp -1 to 1
    ]
)

def transform(examples):
    examples = preprocess(examples)
    return examples


In [59]:
# Create and Split the dataset on Train, Validation and Test Datasets
dataset = HandwrittenAlphabetDataset(handwritten_alphabet, 28, transform)

train,val,test = torch.utils.data.random_split(dataset,[0.8,0.1,0.1])


### A little fnct to show easily images

In [56]:
def show_images(images_tensor:torch.tensor):
    n_dims = images_tensor.dim()
    
    if n_dims in (2,3):
        x_cat = images_tensor

    elif n_dims == 4:
        x_list = [img for img in images_tensor]
        x_cat = torch.cat(x_list,dim=2)
    
    else:
        raise SyntaxError("The dimensions of images_tensor must be between 2 and 4")

    if n_dims != 2:
        if x_cat.shape[0] == 1:
            plt.imshow(x_cat.movedim(0,-1),cmap="gray");
        else:
            plt.imshow(x_cat.movedim(0,-1));
    else:
        plt.imshow(x_cat,cmap="gray");

In [ ]:
type(train[0][0])

torch.Tensor

: 

## Preparing images to the net

In [62]:
#   Create the Dataloader
batch_size = 16

train_dataloader = torch.utils.data.DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val,
    batch_size=batch_size,
    shuffle=True
)
